In [4]:
from networkx.readwrite import json_graph
import json
import numpy as np
import networkx as nx
from collections import defaultdict

### Make train_prefix-G.json of cora dataset

In [6]:
G = nx.read_edgelist('../cora/cora.cites', nodetype=int)

In [7]:
np.random.seed(1)
num_nodes = 2708
rand_indices = np.random.permutation(num_nodes)

In [8]:
nodes_key_list = list(G.nodes())

In [10]:
# number of samples for test, val is used same as pytorch implementation
# https://github.com/williamleif/graphsage-simple
for i, n in enumerate(nodes_key_list):
    if i<1000:
        G.nodes[n]["test"] = True
        G.nodes[n]["val"] = False
    if 1000<=i<1500:
        G.nodes[n]["test"] = False
        G.nodes[n]["val"] = True
    if i>=1500:
        G.nodes[n]["test"] = False
        G.nodes[n]["val"] = False

In [11]:
data = json_graph.node_link_data(G)

In [12]:
with open("data-G.json", mode="w") as f:
    f.write(json.dumps(data))

In [24]:
# This function is used in pytorch implementation
def load_cora():
    num_nodes = 2708
    num_feats = 1433
    feat_data = np.zeros((num_nodes, num_feats))
    labels = np.empty((num_nodes,1), dtype=np.int64)
    node_map = {}
    label_map = {}
    with open("../cora/cora.content") as fp:
        for i,line in enumerate(fp):
            info = line.strip().split()
            feat_data[i,:] = list(map(float, info[1:-1]))
            node_map[info[0]] = i
            if not info[-1] in label_map:
                label_map[info[-1]] = len(label_map)
            labels[i] = label_map[info[-1]]

    adj_lists = defaultdict(set)
    with open("../cora/cora.cites") as fp:
        for i,line in enumerate(fp):
            info = line.strip().split()
            paper1 = node_map[info[0]]
            paper2 = node_map[info[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)
    return feat_data, labels, adj_lists, node_map

In [25]:
feat_data, labels, adj_lists, nodes = load_cora()

In [17]:
with open("data-id_map.json", mode="w") as f:
    f.write(json.dumps(nodes))

In [18]:
labels_reshape = labels.flatten()

In [19]:
n_labels = len(np.unique(labels_reshape))
labels_one_hot = np.eye(n_labels,dtype=int)[labels_reshape]
class_map = {k: list(labels_one_hot[i]) for i,k in enumerate(nodes.keys())}

In [20]:
with open("data-class_map.json", mode="w") as f:
    f.write(json.dumps(class_map,default=str))

In [21]:
np.save('data-feats.npy', feat_data)